<a href="https://colab.research.google.com/github/Brian-2000/A-complete-website-using-laravel/blob/main/LandCoverai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install rasterio
! pip install tqdm
! pip install torch
! pip install torchvision
! pip install torchsummary
! pip install google.colab
! pip install torchmetrics

import rasterio
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from PIL import Image
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 45.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-ma

In [4]:
class PartialCrossEntropyLoss(nn.Module):
    def __init__(self, ignore_index=-100):
        super(PartialCrossEntropyLoss, self).__init__()
        self.ignore_index = ignore_index

    def forward(self, inputs, targets):
        inputs = inputs.permute(0, 2, 3, 1).contiguous()
        inputs = inputs.view(-1, inputs.size(-1))
        targets = targets.view(-1)

        mask = targets != self.ignore_index
        inputs = inputs[mask]
        targets = targets[mask]

        loss = F.cross_entropy(inputs, targets)
        return loss

In [5]:

class LandCoverDataset(Dataset):
    def __init__(self, image_files, mask_files, transform=None, label_percentage=0.1, target_size=(512, 512), num_classes=4):
        self.image_files = image_files
        self.mask_files = mask_files
        self.transform = transform
        self.label_percentage = label_percentage
        self.target_size = target_size
        self.num_classes = num_classes

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        with rasterio.open(self.image_files[idx]) as img_src:
            image = img_src.read([1, 2, 3])  # Read RGB channels
            image = np.transpose(image, (1, 2, 0))  # HWC format
            image = Image.fromarray(image.astype('uint8'), 'RGB')

        with rasterio.open(self.mask_files[idx]) as mask_src:
            mask = mask_src.read(1)  # Read single channel mask
            mask = Image.fromarray(mask.astype('uint8'), 'L')

        # Resize images and masks
        image = transforms.functional.resize(image, self.target_size)
        mask = transforms.functional.resize(mask, self.target_size, interpolation=Image.NEAREST)

        if self.transform:
            image = self.transform(image)

        mask_tensor = torch.tensor(np.array(mask), dtype=torch.long)

        # Clip the mask values to be within the range of the number of classes
        mask_tensor = torch.clamp(mask_tensor, 0, self.num_classes - 1)

        mask_tensor = self.generate_random_labels(mask_tensor)

        return image, mask_tensor

    def generate_random_labels(self, mask):
        labeled_mask = torch.full(mask.shape, -100, dtype=torch.long)
        num_labels = int(self.label_percentage * mask.numel())

        indices = torch.randperm(mask.numel())[:num_labels]
        labeled_mask.view(-1)[indices] = mask.view(-1)[indices]

        return labeled_mask

# Paths to the dataset
image_files = [
    "/content/drive/MyDrive/training_data/N-33-119-C-c-3-3.tif",
    "/content/drive/MyDrive/training_data/N-33-60-D-d-1-2.tif",
    "/content/drive/MyDrive/training_data/N-34-61-B-a-1-1.tif",
    "/content/drive/MyDrive/training_data/N-34-66-C-c-4-3.tif"
]
mask_files = [
    "/content/drive/MyDrive/training_data/N-33-119-C-c-3-3.tif",
    "/content/drive/MyDrive/training_data/N-33-60-D-d-1-2.tif",
    "/content/drive/MyDrive/training_data/N-34-61-B-a-1-1.tif",
    "/content/drive/MyDrive/training_data/N-34-66-C-c-4-3.tif"
]

transform = transforms.Compose([
    transforms.ToTensor(),
])

dataset = LandCoverDataset(image_files, mask_files, transform, label_percentage=0.1, target_size=(512, 512))
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [6]:
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=4):
        super(UNet, self).__init__()
        self.encoder1 = self.double_conv(in_channels, 64)
        self.encoder2 = self.double_conv(64, 128)
        self.encoder3 = self.double_conv(128, 256)
        self.encoder4 = self.double_conv(256, 512)
        self.encoder5 = self.double_conv(512, 1024)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.upconv4 = self.up_conv(1024, 512)
        self.decoder4 = self.double_conv(1024, 512)
        self.upconv3 = self.up_conv(512, 256)
        self.decoder3 = self.double_conv(512, 256)
        self.upconv2 = self.up_conv(256, 128)
        self.decoder2 = self.double_conv(256, 128)
        self.upconv1 = self.up_conv(128, 64)
        self.decoder1 = self.double_conv(128, 64)

        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def double_conv(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def up_conv(self, in_channels, out_channels):
        return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool(enc1))
        enc3 = self.encoder3(self.pool(enc2))
        enc4 = self.encoder4(self.pool(enc3))
        enc5 = self.encoder5(self.pool(enc4))

        dec4 = self.upconv4(enc5)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)

        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)

        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)

        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)

        return self.final_conv(dec1)

# Verify model dimensions
model = UNet()
x = torch.randn((4, 3, 512, 512))  # Batch of 4 images with 3 channels (RGB), size 512x512
output = model(x)
print(output.shape)  # Should be [4, 4, 512, 512] for 4 output channels

torch.Size([4, 4, 512, 512])


In [7]:
class SemiSupervisedLoss(nn.Module):
    def __init__(self, ignore_index=-100):
        super(SemiSupervisedLoss, self).__init__()
        self.ignore_index = ignore_index
        self.cross_entropy = nn.CrossEntropyLoss(ignore_index=ignore_index)

    def forward(self, outputs, labels, unlabeled_outputs=None):
        ce_loss = self.cross_entropy(outputs, labels)
        if unlabeled_outputs is not None:
            pseudo_labels = torch.argmax(unlabeled_outputs, dim=1)
            mask = labels != self.ignore_index
            pseudo_labels[mask] = labels[mask]
            unsupervised_loss = F.cross_entropy(outputs, pseudo_labels)
            return ce_loss + 0.1 * unsupervised_loss
        return ce_loss

In [8]:
import torch
def train_model(dataloader, model, criterion, optimizer, num_epochs=25):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

def evaluate_model(dataloader, model):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    print(f"Validation Loss: {total_loss/len(dataloader)}")

# Main training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet().to(device)
criterion = SemiSupervisedLoss(ignore_index=-100)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create the dataloader
dataset = LandCoverDataset(image_files, mask_files, transform, label_percentage=0.1, target_size=(512, 512))
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
train_model(dataloader, model, criterion, optimizer, num_epochs=25)

100%|██████████| 1/1 [00:21<00:00, 21.51s/it]


Epoch 1/25, Loss: 1.3795219659805298


100%|██████████| 1/1 [00:10<00:00, 10.77s/it]


Epoch 2/25, Loss: 1.2791986465454102


100%|██████████| 1/1 [00:10<00:00, 10.82s/it]


...

Epoch 23/25, Loss: 0.38962435722351074


100%|██████████| 1/1 [00:10<00:00, 10.37s/it]


Epoch 24/25, Loss: 0.3770003616809845


100%|██████████| 1/1 [00:10<00:00, 10.32s/it]

Epoch 25/25, Loss: 0.36391326785087585
